In [1]:
import requests
import json
import csv
import datetime

pos=1
USD_BTC=[]
USD_BTC_buy=[]
USD_BTC_sell=[]

def write_csv(new_line):
    with open("common.csv", "a" ) as fout:
        writer=csv.writer(fout)
        writer.writerow((new_line))
        
fees={"trade":{'btc-e': 0.002,
               'bitfinex': 0.002,
               'poloniex': 0.0025,
               'bittrex': 0.0025,
               'kraken': 0.0026,
               'hitbtc': 0.001,
               'yobit': 0.002,
               'exmo': 0.002,
               'livecoin': 0.002,
               "cexio": 0.002},      
      "trans_BTC":{'btc-e': 0.001,
                   'bitfinex': 0.0004,
                   'poloniex': 0.0001,
                   'bittrex': 0.001,
                   'kraken': 0.001,
                   'hitbtc': 0.00085,
                   'yobit': 0.0015,
                   'exmo': 0.001,
                   'livecoin': 0.001,
                   'cexio': 0.001},
      'USD_deposit':{'btc-e':{'rate':0.005,
                              'min':20,
                              'fix':0},      
                    'bitfinex':{'rate':1,    # Не принимают на ввод баксы. стоит заградительный тариф.
                                'min':1000,  # Не принимают на ввод баксы. стоит заградительный тариф.
                                'fix':1000}, # Не принимают на ввод баксы. стоит заградительный тариф.                    
                    'poloniex':{'rate':1,    # Не принимают на ввод баксы. стоит заградительный тариф.
                                'min':1000,  # Не принимают на ввод баксы. стоит заградительный тариф.
                                'fix':1000}, # Не принимают на ввод баксы. стоит заградительный тариф.                    
                    'bittrex':{'rate':1,     # Не принимают на ввод баксы. стоит заградительный тариф.
                                'min':1000,  # Не принимают на ввод баксы. стоит заградительный тариф.
                                'fix':1000}, # Не принимают на ввод баксы. стоит заградительный тариф.                     
                    'kraken':{'rate':0,     
                              'min':0,  
                              'fix':10},                     
                    'hitbtc':{'rate':0.01,     
                              'min':6,  
                              'fix':0},                    
                    'yobit':{'rate':0.005,     
                             'min':20,  
                             'fix':0},                     
                    'exmo':{'rate':0.005,     
                             'min':20,  
                             'fix':0},                     
                    'livecoin':{'rate':0.005,     
                                'min':20,  
                                'fix':0},                     
                    'cexio':{'rate':0,     
                             'min':0,  
                             'fix':0},
                     'bank':{'rate':0.01,     
                             'min':15,  
                             'fix':0}},                    
      'USD_withdraw':{'btc-e':{'rate':0.025,
                               'min':0,
                               'fix':20},
                      'bitfinex':{'rate':0.001,    
                                  'min':20,  
                                  'fix':0},
                      'poloniex':{'rate':1,    # Не принимают на вывод баксы. стоит заградительный тариф.
                                  'min':1000,  # Не принимают на вывод баксы. стоит заградительный тариф.
                                  'fix':1000}, # Не принимают на вывод баксы. стоит заградительный тариф.
                      'bittrex':{'rate':1,     # Не принимают на вывод баксы. стоит заградительный тариф.
                                'min':1000,    # Не принимают на вывод баксы. стоит заградительный тариф.
                                'fix':1000},   # Не принимают на вывод баксы. стоит заградительный тариф.
                      'kraken':{'rate':0,     
                                'min':0,  
                                'fix':60},
                      'hitbtc':{'rate':0.02,     
                                'min':35,  
                                'fix':0},
                      'yobit':{'rate':0.05,     
                               'min':0,  
                               'fix':0},
                      'exmo':{'rate':0.03,     
                              'min':60,  
                              'fix':0},
                      'livecoin':{'rate':0.015,     
                                  'min':0,  
                                  'fix':9},
                      'cexio':{'rate':0,     
                               'min':0,  
                               'fix':3.8},
                      'bank':{'rate':0,     
                              'min':0,  
                              'fix':0}}}

In [2]:
# btc-e
try:
    base_url="https://btc-e.nz/api/3/"
    pairs="btc_usd"

    ticker_url=base_url+"ticker/"+pairs
    depth_url=base_url+"depth/"+pairs

    html_ticker=requests.get(ticker_url).text
    html_depth=requests.get(depth_url).text

    ticker_data=json.loads(html_ticker)
    depth_data=json.loads(html_depth)    

    for i in ticker_data:
        q=0
        w=[]
        e=[]
        for j in depth_data[i]["bids"]:
            q=q+j[1]
            if q>pos:
                w.append(j)
            if q<pos:
                e.append(j)      
        r=0
        t=0
        for j in e:
            r=j[0]*j[1]+r
            t=j[1]+t    
        r=w[0][0]*(pos-t)+r
        bids=r/pos


        q=0
        w=[]
        e=[]
        for j in depth_data[i]["asks"]:
            q=q+j[1]
            if q>pos:
                w.append(j)
            if q<pos:
                e.append(j)    
        r=0
        t=0
        for j in e:
            r=j[0]*j[1]+r
            t=j[1]+t    
        r=w[0][0]*(pos-t)+r
        asks=r/pos    
    
        line=[str(datetime.datetime.now())[:-7], "btc-e", "USD_BTC", str(ticker_data[i]["buy"]).replace(".",","),
              str(ticker_data[i]["sell"]).replace(".",","), str(asks).replace(".",","), str(bids).replace(".",",")]
        
        print line 
        USD_BTC.append(line)
        
        #write_csv(line)
except:
    print "btc-e failed"
    pass    
    
# bitfinex
try:
    base_url="https://api.bitfinex.com/v1/"
    pairs=open('pairs_btc_usd.txt').read()
    pairs=pairs.lower().replace("_","").split("\n")

    symbol_url=base_url+"symbols"
    html_symbol=requests.get(symbol_url).text
    symbol_data=json.loads(html_symbol)
    #print symbol_data

    for i in symbol_data:
        if i in pairs:
            ticker_url=base_url+"pubticker/"+i
            html_ticker=requests.get(ticker_url).text
            ticker_data=json.loads(html_ticker)        

            depth_url=base_url+"book/"+i
            html_depth=requests.get(depth_url).text
            depth_data=json.loads(html_depth)

            q=0
            w=[]
            e=[]
            for j in depth_data["bids"]:
                q=q+float(j["amount"])
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)      
            r=0
            t=0
            for j in e:
                r=float(j["price"])*float(j["amount"])+r
                t=float(j["amount"])+t    
            r=float(w[0]["price"])*(pos-t)+r
            bids=r/pos
    
            q=0
            w=[]
            e=[]
            for j in depth_data["asks"]:
                q=q+float(j["amount"])
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)    
            r=0
            t=0
            for j in e:
                r=float(j["price"])*float(j["amount"])+r
                t=float(j["amount"])+t    
            r=float(w[0]["price"])*(pos-t)+r
            asks=r/pos     
    
            line=[str(datetime.datetime.now())[:-7], "bitfinex", "USD_BTC", str(ticker_data["ask"]).replace(".",","), 
                  str(ticker_data["bid"]).replace(".",","), str(asks).replace(".",","), str(bids).replace(".",",")]
       
            print line 
            USD_BTC.append(line)
        
            #write_csv(line)
except:
    print "bitfinex failed"
    pass
        
# poloniex
try:
    base_url="https://poloniex.com/public?command=return"
    pairs=open('pairs_btc_usd.txt').read().split("\n")

    ticker_url=base_url+"Ticker"
    html_ticker=requests.get(ticker_url).text
    ticker_data=json.loads(html_ticker)

    for i in ticker_data:
        if i in pairs:
            depth_url=base_url+"OrderBook&currencyPair="+i+"&depth=20"    
            html_depth=requests.get(depth_url).text
            depth_data=json.loads(html_depth)
    
            q=0
            w=[]
            e=[]
            for j in depth_data["bids"]:    
                q=q+j[1]
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)  
            r=0
            t=0
            for j in e:
                r=float(j[0])*j[1]+r
                t=j[1]+t    
            r=float(w[0][0])*(pos-t)+r
            bids=r/pos
    
            q=0
            w=[]
            e=[]
            for j in depth_data["asks"]:    
                q=q+j[1]
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)  
            r=0
            t=0
            for j in e:
                r=float(j[0])*j[1]+r
                t=j[1]+t    
            r=float(w[0][0])*(pos-t)+r
            asks=r/pos        
                
            line=[str(datetime.datetime.now())[:-7], "poloniex", "USD_BTC", str(ticker_data[i]["lowestAsk"]).replace(".",","), 
                  str(ticker_data[i]["highestBid"]).replace(".",","), str(asks).replace(".",","), str(bids).replace(".",",")]
        
            print line 
            USD_BTC.append(line)
        
            #write_csv(line)
except:
    print "poloniex failed"
    pass
        
# bittrex
try:
    base_url="https://bittrex.com/api/v1.1/public/"
    pairs=open('pairs_btc_usd.txt').read()
    pairs=pairs.replace("_","-").split("\n")

    symbol_url=base_url+"getmarkets"
    html_symbol=requests.get(symbol_url).text
    symbol_data=json.loads(html_symbol)

    for i in symbol_data['result']:
        if i['MarketName'] in pairs:
            ticker_url=base_url+"getticker?market="+i['MarketName']
            html_ticker=requests.get(ticker_url).text
            ticker_data=json.loads(html_ticker)
    
            depth_url=base_url+"getorderbook?market="+i['MarketName']+"&type=both&depth=20"   
            html_depth=requests.get(depth_url).text
            depth_data=json.loads(html_depth)
    
            try:
                q=0
                w=[]
                e=[]
                for j in depth_data['result']['buy']:
                    q=q+j['Quantity']
                    if q>pos:
                        w.append(j)
                    if q<pos:
                        e.append(j) 
                r=0
                t=0
                for j in e:
                    r=j["Rate"]*j['Quantity']+r
                    t=j['Quantity']+t    
                r=w[0]["Rate"]*(pos-t)+r
                bids=r/pos
            except:
                bids=ticker_data['result']["Bid"]
        
            try:
                q=0
                w=[]
                e=[]
                for j in depth_data['result']['sell']:
                    q=q+j['Quantity']
                    if q>pos:
                        w.append(j)
                    if q<pos:
                        e.append(j) 
                r=0
                t=0
                for j in e:
                    r=j["Rate"]*j['Quantity']+r
                    t=j['Quantity']+t    
                r=w[0]["Rate"]*(pos-t)+r
                asks=r/pos
            except:
                asks=ticker_data['result']["Ask"]
        
            line = [str(datetime.datetime.now())[:-7], 'bittrex', "USD_BTC", str(ticker_data['result']["Ask"]).replace(".",","),
                    str(ticker_data['result']["Bid"]).replace(".",","), str(asks).replace(".",","), str(bids).replace(".",",")]
            
            print line 
            USD_BTC.append(line)
        
            #write_csv(line)
except:
    print "bittrex failed"
    pass
        
# kraken
try:
    base_url="https://api.kraken.com/0/public/"

    pairs=open('pairs_btc_usd.txt').read()
    pairs=pairs.replace("_","").replace("BTC","XBT").split("\n")

    symbol_url=base_url+"AssetPairs"
    html_symbol=requests.get(symbol_url).text
    symbol_data=json.loads(html_symbol)

    symbols=[]
    z=0
    for i in symbol_data['result']:   
        symbols.append(symbol_data['result'][i]["altname"])    

    symbols=str(symbols).replace("[","").replace("]",'').replace("u",'').replace(" ","").replace("'","")

    ticker_url=base_url+"Ticker?pair="+symbols
    html_ticker=requests.get(ticker_url).text
    ticker_data=json.loads(html_ticker)

    for i in ticker_data['result']:
        if symbol_data['result'][i]["altname"] in pairs:
            depth_url=base_url+"Depth?pair="+symbol_data['result'][i]["altname"]+'&count=20'
            html_depth=requests.get(depth_url).text
            depth_data=json.loads(html_depth)
    
            try:
                q=0
                w=[]
                e=[]
                for j in depth_data['result'][i]['bids']:    
                    q=q+float(j[1])
                    if q>pos:
                        w.append(j)
                    if q<pos:
                        e.append(j)  
                r=0
                t=0
                for j in e:
                    r=float(j[0])*j[1]+r
                    t=j[1]+t    
                r=float(w[0][0])*(pos-t)+r
                bids=r/pos        
            except:
                bids=ticker_data['result'][i]['b'][0]        
        
            try:
                q=0
                w=[]
                e=[]
                for j in depth_data['result'][i]['asks']:    
                    q=q+float(j[1])
                    if q>pos:
                        w.append(j)
                    if q<pos:
                        e.append(j)  
                r=0
                t=0
                for j in e:
                    r=float(j[0])*j[1]+r
                    t=j[1]+t    
                r=float(w[0][0])*(pos-t)+r
                asks=r/pos        
            except:
                asks=ticker_data['result'][i]['a'][0]
        
            line=[str(datetime.datetime.now())[:-7], "kraken", "USD_BTC", str(ticker_data['result'][i]['a'][0]).replace(".",","),
                  str(ticker_data['result'][i]['b'][0]).replace(".",","), str(asks).replace(".",","), str(bids).replace(".",",")]
        
            print line 
            USD_BTC.append(line)
        
            #write_csv(line)
except:
    print "kraken failed"
    pass
        
# hitbtc
try:
    base_url="https://api.hitbtc.com/api/1/public/"
    pairs=open('pairs_btc_usd.txt').read()
    pairs=pairs.replace("_","").split("\n")

    ticker_url=base_url+"ticker"
    html_ticker=requests.get(ticker_url).text
    ticker_data=json.loads(html_ticker)

    for i in ticker_data:
        if i in pairs:
            depth_url=base_url+i+"/orderbook?format_price=number&format_amount=number"
            html_depth=requests.get(depth_url).text
            depth_data=json.loads(html_depth)
    
            q=0
            w=[]
            e=[]
            for j in depth_data["bids"]:
                q=q+j[1]
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)      
            r=0
            t=0
            for j in e:
                r=j[0]*j[1]+r
                t=j[1]+t    
            r=w[0][0]*(pos-t)+r
            bids=r/pos
    
            q=0
            w=[]
            e=[]
            for j in depth_data["asks"]:
                q=q+j[1]
                if q>pos:
                    w.append(j)
                if q<pos:
                    e.append(j)      
            r=0
            t=0
            for j in e:
                r=j[0]*j[1]+r
                t=j[1]+t    
            r=w[0][0]*(pos-t)+r
            asks=r/pos
    
            line=[str(datetime.datetime.now())[:-7], "hitbtc", "USD_BTC", str(ticker_data[i]["ask"]).replace(".",","), 
                  str(ticker_data[i]["bid"]).replace(".",","), str(asks).replace(".",","), str(bids).replace(".",",")]
        
            print line 
            USD_BTC.append(line)
        
            #write_csv(line)
except:
    print "hitbtc failed"
    pass
        
# yobit
try:
    base_url="https://yobit.net/api/3/"
    pairs=open('pairs_btc_usd.txt').read().split("\n")

    symbol_url=base_url+"info"
    html_symbol=requests.get(symbol_url).text
    symbol_data=json.loads(html_symbol)

    for i in symbol_data['pairs']:
        if i.upper() in pairs:
            ticker_url=base_url+"ticker/"+i
            html_ticker=requests.get(ticker_url).text
            ticker_data=json.loads(html_ticker)
    
            depth_url=base_url+"depth/"+i+"?limit=20"
            html_depth=requests.get(depth_url).text
            depth_data=json.loads(html_depth)
    
            try:
                q=0
                w=[]
                e=[]
                for j in depth_data[i]["bids"]:
                    q=q+j[1]
                    if q>pos:
                        w.append(j)
                    if q<pos:
                        e.append(j)      
                r=0
                t=0
                for j in e:
                    r=j[0]*j[1]+r
                    t=j[1]+t    
                r=w[0][0]*(pos-t)+r
                bids=r/pos
            except:
                bids=ticker_data[i]['buy']

            try:
                q=0
                w=[]
                e=[]
                for j in depth_data[i]["asks"]:
                    q=q+j[1]
                    if q>pos:
                        w.append(j)
                    if q<pos:
                        e.append(j)    
                r=0
                t=0
                for j in e:
                    r=j[0]*j[1]+r
                    t=j[1]+t    
                r=w[0][0]*(pos-t)+r
                asks=r/pos
            except:
                asks=ticker_data[i]['sell']
    
            line=[str(datetime.datetime.now())[:-7], 'yobit', "USD_BTC", str(ticker_data[i]['sell']).replace(".",","), 
                  str(ticker_data[i]['buy']).replace(".",","), str(asks).replace(".",","), str(bids).replace(".",",")]
        
            print line 
            USD_BTC.append(line)
        
            #write_csv(line)
except:
    print "yobit failed"
    pass
        
# exmo
try:
    base_url="https://api.exmo.com/v1/"

    ticker_url=base_url+"ticker/"
    html_ticker=requests.get(ticker_url).text
    ticker_data=json.loads(html_ticker)

    #for i in ticker_data:
    depth_url=base_url+"order_book/?pair=BTC_USD&limit=20"
    html_depth=requests.get(depth_url).text
    depth_data=json.loads(html_depth)
    
    try:
        q=0
        w=[]
        e=[]
        for j in depth_data["BTC_USD"]['bid']:    
            q=q+float(j[1])
            if q>pos:
                w.append(j)
            if q<pos:
                e.append(j)  
        r=0
        t=0
        for j in e:
            r=float(j[0])*float(j[1])+r
            t=float(j[1])+t    
        r=float(w[0][0])*(pos-t)+r
        bids=r/pos
    except:
        bids=ticker_data["BTC_USD"]['buy_price']
        
    try:
        q=0
        w=[]
        e=[]
        for j in depth_data["BTC_USD"]['ask']:    
            q=q+float(j[1])
            if q>pos:
                w.append(j)
            if q<pos:
                e.append(j)  
        r=0
        t=0
        for j in e:
            r=float(j[0])*float(j[1])+r
            t=float(j[1])+t    
        r=float(w[0][0])*(pos-t)+r
        asks=r/pos
    except:
        asks=ticker_data["BTC_USD"]['sell_price']
        
    line=[str(datetime.datetime.now())[:-7], "exmo", "USD_BTC", str(ticker_data["BTC_USD"]['sell_price']).replace(".",","), 
          str(ticker_data["BTC_USD"]['buy_price']).replace(".",","), str(asks).replace(".",","), str(bids).replace(".",",")]
    
    print line 
    USD_BTC.append(line)

    #write_csv(line)
except:
    print "exmo failed"
    pass

#livecoin
try:
    base_url="https://api.livecoin.net/exchange/"

    ticker_url=base_url+"ticker?currencyPair=BTC/USD"
    depth_url=base_url+"order_book?currencyPair=BTC/USD"

    html_ticker=requests.get(ticker_url).text
    html_depth=requests.get(depth_url).text

    ticker_data=json.loads(html_ticker)
    depth_data=json.loads(html_depth)

    #for i in ticker_data:
    try:
        q=0
        w=[]
        e=[]
        for j in depth_data["bids"]:
            q=q+float(j[1])
            if q>pos:
                w.append(j)
            if q<pos:
                e.append(j)     
            
        r=0
        t=0
        for j in e:
            r=float(j[0])*float(j[1])+r
            t=float(j[1])+t    
        r=float(w[0][0])*(pos-t)+r
        bids=r/pos 
    
    except:
        bids=ticker_data["best_bid"]
        
    try:
        q=0
        w=[]
        e=[]
        for j in depth_data["asks"]:
            q=q+float(j[1])
            if q>pos:
                w.append(j)
            if q<pos:
                e.append(j)     
            
        r=0
        t=0
        for j in e:
            r=float(j[0])*float(j[1])+r
            t=float(j[1])+t    
        r=float(w[0][0])*(pos-t)+r
        asks=r/pos      
    
    except:
        asks=ticker_data["best_ask"]    
        
        
    line = [str(datetime.datetime.now())[:-7], "livecoin", "USD_BTC", str(ticker_data["best_ask"]).replace(".",","), 
            str(ticker_data["best_bid"]).replace(".",","), str(asks).replace(".",","), str(bids).replace(".",",")]
    
    print line 
    USD_BTC.append(line)
    
    #write_csv(line)
    
except:
    print "livecoin failed"
    pass

# cexio
try:
    base_url="https://cex.io/api/"

    ticker_url=base_url+"ticker/BTC/USD"
    html_ticker=requests.get(ticker_url).text
    ticker_data=json.loads(html_ticker)

    #for i in ticker_data["data"]:
    depth_url=base_url+"order_book/BTC/USD/?depth=20"
    html_depth=requests.get(depth_url).text
    depth_data=json.loads(html_depth)
    
    q=0
    w=[]
    e=[]
    for j in depth_data['bids']:    
        q=q+float(j[1])
        if q>pos:
            w.append(j)
        if q<pos:
            e.append(j)  
    r=0
    t=0
    for j in e:
        r=float(j[0])*j[1]+r
        t=j[1]+t    
    r=float(w[0][0])*(pos-t)+r
    bids=r/pos
    
    q=0
    w=[]
    e=[]
    for j in depth_data['asks']:    
        q=q+float(j[1])
        if q>pos:
            w.append(j)
        if q<pos:
            e.append(j)  
    r=0
    t=0
    for j in e:
        r=float(j[0])*j[1]+r
        t=j[1]+t    
    r=float(w[0][0])*(pos-t)+r
    asks=r/pos
    
    line=[str(datetime.datetime.now())[:-7], "cexio", "USD_BTC", str(ticker_data["ask"]).replace(".",","),
          str(ticker_data["bid"]).replace(".",","), str(asks).replace(".",","), str(bids).replace(".",",")]
    
    print line   
    USD_BTC.append(line)
    
    #write_csv(line)
except:
    print "cexio failed"
    pass


for i in USD_BTC:
    "______________________________________________________________________________"
    # котировка на продажу
    buy_price = float(str(i[5]).replace(",","."))
    
    # Комиссия банка за перевод на биржу 1
    trans_comis_bank_float = fees["USD_deposit"]["bank"]["rate"]*float(str(i[5]).replace(",","."))
    if trans_comis_bank_float < fees["USD_deposit"]["bank"]["min"]:
        trans_comis_bank_float = fees["USD_deposit"]["bank"]["min"]
    trans_comis_bank = trans_comis_bank_float + fees["USD_deposit"]["bank"]["fix"]
    
    # комиссия биржы 1 за ввод
    trans_comis_deposit_float = fees["USD_deposit"][i[1]]["rate"]*float(str(i[5]).replace(",","."))
    if trans_comis_deposit_float < fees["USD_deposit"][i[1]]["min"]:
        trans_comis_deposit_float = fees["USD_deposit"][i[1]]["min"]
    trans_comis_deposit = trans_comis_deposit_float + fees["USD_deposit"][i[1]]["fix"]
    
    # торговая комиссия за покупку актива
    trade_comis_buy = float(str(i[5]).replace(",","."))*fees["trade"][i[1]]*pos
    
    # за перевод купленного актива с биржы 1 на биржу 2    
    trans_comis_buy = fees["trans_BTC"][i[1]]*(float(str(i[5]).replace(",","."))+float(str(i[6]).replace(",",".")))/2
    
    # всего комиссий при покупке
    total_comis_buy = trans_comis_bank + trans_comis_deposit + trade_comis_buy + trans_comis_buy    
    
    USD_BTC_buy.append([float(str(i[5]).replace(",","."))+total_comis_buy, i[1],
                       float(str(i[5]).replace(",",".")), total_comis_buy])
    "________________________________________________________________________________"    
    # котировка на покупку
    buy_price = float(str(i[6]).replace(",","."))
    
    # торговая комиссия за продажу актива
    trade_comis_sell = float(str(i[6]).replace(",","."))*fees["trade"][i[1]]*pos
    
    # комиссия биржы 2 за вывод
    trans_comis_withdraw_float = fees["USD_withdraw"][i[1]]["rate"]*float(str(i[6]).replace(",","."))
    if trans_comis_withdraw_float < fees["USD_withdraw"][i[1]]["min"]:
        trans_comis_withdraw_float = fees["USD_withdraw"][i[1]]["min"]
    trans_comis_withdraw = trans_comis_withdraw_float + fees["USD_withdraw"][i[1]]["fix"]
    
    # Комиссия банка за вывод с биржы 2
    trans_comis_bank2_float = fees["USD_withdraw"]["bank"]["rate"]*float(str(i[6]).replace(",","."))
    if trans_comis_bank2_float < fees["USD_withdraw"]["bank"]["min"]:
        trans_comis_bank2_float = fees["USD_withdraw"]["bank"]["min"]
    trans_comis_bank2 = trans_comis_bank2_float + fees["USD_withdraw"]["bank"]["fix"]
    
    # всего комиссий при продаже
    total_comis_sell = trade_comis_sell + trans_comis_withdraw + trans_comis_bank2
                 
    USD_BTC_sell.append([float(str(i[6]).replace(",","."))-total_comis_sell, i[1],
                        float(str(i[6]).replace(",",".")), total_comis_sell])
    "________________________________________________________________________________"


new_line=[str(datetime.datetime.now())[:-7], "USD_BTC", 
         str(min(zip(*zip(*USD_BTC_buy)))[2]).replace(".",","), # котировка на покупку
         str(min(zip(*zip(*USD_BTC_buy)))[3]).replace(".",","), # размер общих комиссий на покупку
         min(zip(*zip(*USD_BTC_buy)[0:2]))[1], # биржа покупки
         str(min(zip(*zip(*USD_BTC_buy)[0:2]))[0]).replace(".",","), # эффективная цена покупки
         str(max(zip(*zip(*USD_BTC_sell)[0:2]))[0]).replace(".",","), # эффективная цена продажи
         max(zip(*zip(*USD_BTC_sell)[0:2]))[1],  # биржа продажи
         str(max(zip(*zip(*USD_BTC_sell)))[3]).replace(".",","), # размер общих комиссий на продажу
         str(max(zip(*zip(*USD_BTC_sell)))[2]).replace(".",","), # котировка на продажу 
         str((max(zip(*zip(*USD_BTC_sell)[0:2]))[0]/min(zip(*zip(*USD_BTC_buy)[0:2]))[0]-1)*100).replace(".",",")+"%",
         str(100*(max(zip(*zip(*USD_BTC_sell)))[2]-min(zip(*zip(*USD_BTC_buy)))[2]-min(zip(*zip(*USD_BTC_buy)))[3]-max(zip(*zip(*USD_BTC_sell)))[3])/(min(zip(*zip(*USD_BTC_buy)))[2]+min(zip(*zip(*USD_BTC_buy)))[3]+max(zip(*zip(*USD_BTC_sell)))[3])).replace(".",",")+"%"]

print new_line
write_csv(new_line)

['2017-06-23 15:29:56', 'btc-e', 'USD_BTC', '2676,234', '2669,005', '2676,67951572', '2669,005']
['2017-06-23 15:29:58', 'bitfinex', 'USD_BTC', '2677,6', '2671,7', '2677,93051944', '2677,6']
['2017-06-23 15:30:11', 'kraken', 'USD_BTC', '2697,92000', '2694,01600', '2697,92000', '2692,0']
['2017-06-23 15:30:11', 'hitbtc', 'USD_BTC', '2681,95', '2678,04', '2691,8573', '2664,1905']
['2017-06-23 15:30:12', 'yobit', 'USD_BTC', '2725,999999', '2712,78000004', '2730,90214806', '2707,75067744']
['2017-06-23 15:30:12', 'exmo', 'USD_BTC', '2759,74', '2741,11005', '2764,38460547', '2740,98847292']
['2017-06-23 15:30:14', 'livecoin', 'USD_BTC', '2705,0', '2670,02', '2720,30582371', '2624,61018592']
['2017-06-23 15:30:14', 'cexio', 'USD_BTC', '2818,41', '2816,4596', '2819,01877244', '2816,38671655']
['2017-06-23 15:30:14', 'USD_BTC', '2676,67951572', '54,7929964465', 'btc-e', '2731,47251217', '2806,95394312', 'cexio', '9,4327734331', '2816,38671655', '2,76339705467%', '2,75388687624%']
